Denis Kirbaba, group R3338

Bonjour! 

The homework assignment will require you to build several machine learning models and answer questions equivalent to an interview for Data Scientist/Machine Learning Engineer and similar positions.

**Soft deadline** : June 18 23-59 with the option to "free" edits for the next 3 days after I check the homework,

**Hard deadline** : June 25th 23-59 with 50% loss of points

**Task 1** **(40 points)**

*Build machine learning models for the classification problem, namely: k-nearest neighbors, SVM, Parzen window method, Perceptron.*

You can choose any dataset to work with, for example:

Spambase Data Set : https://archive.ics.uci.edu/ml/datasets/Spambase 

Breast Cancer Data Set : https://archive.ics.uci.edu/ml/datasets/breast+cancer

Wine Data Set : https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine 

You can find a dataset you like in the UCI Repository (the first two links). 

There are no clear requirements for model building, you want to see structure, sequence of actions and *always* analysis: what worked, what didn't work, what you expected to see as a result, what was the result and what conclusions you can draw from the model, thoughts about how you could improve the model for more accurate metrics. 

As metrics you can take any metrics that we went through, *it is desirable to try on one model about two or three different metrics* to be able to make some comparison. 

At the beginning, let's build the necessary classification models classes with a standard machine learning model interface:

In [128]:
from collections import Counter

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

class KNN:
    def __init__(self, k=3):
        self.k = k
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)
    def _predict(self, x):
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

Implement SVM classification algorithm from sklearn library

In [129]:
from sklearn.svm import SVC

In [130]:
import numpy as np

class ParzenWindowClassifier():
    def __init__(self, h=1):
        self.h = h
    def gaussian_kernel(self, x):
        return (1/np.sqrt(2*np.pi*self.h**2)) * np.exp(-(x**2)/(2*self.h**2))
    def parzen_window(self, x, X):
        n_samples = X.shape[0]
        likelihoods = np.zeros(n_samples)
        for i in range(n_samples):
            dist = np.linalg.norm(x - X[i,:])
            likelihoods[i] = self.gaussian_kernel(dist)
        return np.mean(likelihoods)
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.classes = np.unique(y)
    def predict(self, X):
        n_samples = X.shape[0]
        y_pred = np.zeros(n_samples)
        for i in range(n_samples):
            posteriors = np.zeros(len(self.classes))
            for j in range(len(self.classes)):
                class_mask = (self.y_train == self.classes[j])
                X_class = self.X_train[class_mask]
                posteriors[j] = self.parzen_window(X[i,:], X_class)
            y_pred[i] = self.classes[np.argmax(posteriors)]
        return y_pred

In [159]:
class Perceptron:
    def __init__(self, learning_rate=0.01, n_iterations=1000, l1_penalty=0, l2_penalty=0):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.l1_penalty = l1_penalty
        self.l2_penalty = l2_penalty
        self.weights = None
        self.bias = None
    def fit(self, X, y):
        n_samples, n_features = X.shape
         # Initialize weights and bias to zero
        self.weights = np.zeros(n_features)
        self.bias = 0
         # Train the model
        for i in range(self.n_iterations):
            for j in range(n_samples):
                # Calculate the predicted value
                y_pred = np.dot(self.weights, X[j]) + self.bias
                 # Update the weights and bias if the prediction is incorrect
                if y[j]*y_pred <= 0:
                    self.weights += self.learning_rate * y[j] * X[j]
                    self.bias += self.learning_rate * y[j]
             # Apply L1 regularization
            if self.l1_penalty > 0:
                self.weights = self.weights - self.learning_rate * self.l1_penalty * np.sign(self.weights)
             # Apply L2 regularization
            if self.l2_penalty > 0:
                self.weights = self.weights - self.learning_rate * self.l2_penalty * self.weights
    def predict(self, X):
        # Calculate the predicted values for the input data
        y_pred = np.dot(X, self.weights) + self.bias
        return np.where(y_pred > 0, 1, -1)

Now choose a dataset on which we will test models and metrics on which to evaluate the performance.
Dataset: Breast Cancer Data Set : https://archive.ics.uci.edu/ml/datasets/breast+cancer
Metrics.
For the Breast Cancer Data Set, which is a binary classification problem, choose three standard ml metrics:
 1. Accuracy: This metric measures the proportion of correctly classified samples. It's a good metric to use when the classes are balanced, which we have in our dataset.
 2. Precision: This metric measures the proportion of true positives among the predicted positives. In the context of breast cancer classification, precision is the proportion of patients who are predicted to have breast cancer and actually have breast cancer. It's a useful metric when the cost of a false positive is high, such as unnecessary biopsies or surgeries.
 3. Recall: This metric measures the proportion of true positives among the actual positives. In the context of breast cancer classification, recall is the proportion of patients who have breast cancer and are correctly identified as having breast cancer. It's a useful metric when the cost of a false negative is high, such as missed diagnoses that delay treatment.

Prepare the dataset

In [132]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Load the dataset into a pandas dataframe:

In [133]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data"
names = ['Class', 'age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat']
df = pd.read_csv(url, names=names)

Remove missing values:

In [134]:
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)

Encode categorical features:

In [135]:
encoder = LabelEncoder()
df['Class'] = encoder.fit_transform(df['Class'])
df['age'] = encoder.fit_transform(df['age'])
df['menopause'] = encoder.fit_transform(df['menopause'])
df['tumor_size'] = encoder.fit_transform(df['tumor_size'])
df['inv_nodes'] = encoder.fit_transform(df['inv_nodes'])
df['node_caps'] = encoder.fit_transform(df['node_caps'])
df['deg_malig'] = encoder.fit_transform(df['deg_malig'])
df['breast'] = encoder.fit_transform(df['breast'])
df['breast_quad'] = encoder.fit_transform(df['breast_quad'])
df['irradiat'] = encoder.fit_transform(df['irradiat'])

Split the dataset into training and testing sets:

In [136]:
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Scale the feature matrix using StandardScaler:

In [137]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [138]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

Let's train and evaluate our methods:

In [139]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

Find an optimal k hyper-parameter for kNN classifier.

In [145]:
# Test each k value and record the results
results = []
for k in k_values:
    clf = KNN(k=k)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    results.append((k, accuracy, recall, precision))

# Print the results
print('Results:')
print('K\tAccuracy\tRecall\tPrecision')
for k, accuracy, recall, precision in results:
    print(f'{k}\t{accuracy:.3f}\t\t{recall:.3f}\t{precision:.3f}')

# Find the optimal k value based on accuracy
optimal_k = max(results, key=lambda x: x[1])[0]
print(f'Optimal k value: {optimal_k}')

Results:
K	Accuracy	Recall	Precision
1	0.625		0.421	0.444
4	0.679		0.368	0.538
7	0.679		0.316	0.545
10	0.732		0.368	0.700
13	0.679		0.211	0.571
16	0.732		0.316	0.750
19	0.732		0.263	0.833
22	0.732		0.263	0.833
25	0.750		0.263	1.000
28	0.732		0.211	1.000
31	0.714		0.158	1.000
34	0.714		0.158	1.000
37	0.696		0.105	1.000
40	0.696		0.105	1.000
43	0.679		0.053	1.000
46	0.679		0.053	1.000
49	0.679		0.053	1.000
Optimal k value: 25


To analyze the results, look at the accuracy, recall, and precision scores for each k value. In general, we want all three scores to be as high as possible. Looking at the results, we can see that the accuracy is highest for k=25, recall is highest for k=1 and precision scores is highest for k>25.
Hish precision means that model better for predicting benign tumors, while high recall is better for predicting malignant tumors (since recall is high).
 
Overall, the KNN classifier performs bad on this dataset, with accuracy=0.75, recall=0.263 and precision=1 for optimal k value. There is still room for improvement, as the scores could be higher.
To improve performance of this method we can
1. try other distance metrics besides Euclidean distance;
2. use feature selection or dimensionality reduction techniques to reduce the number of features used;
3. collect more data to improve the accuracy of the model.

Evaluate SVM classifier. Use gridsearch to find optimal hyper-parameters.

In [150]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf', 'sigmoid', 'poly'], 'coef0': [0, 0.1, 1, 10]}
# Perform GridSearchCV to find optimal hyperparameters
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(X_train, y_train)
# Print best hyperparameters
print("Best hyperparameters:", grid.best_params_)
# Predict on test set using best hyperparameters
y_pred = grid.predict(X_test)
# Calculate accuracy, recall, and precision
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
# Print metrics
print("Accuracy:", acc)
print("Recall:", rec)
print("Precision:", pre)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
[CV 1/5] END C=0.1, coef0=0, gamma=1, kernel=rbf;, score=0.711 total time=   0.0s
[CV 2/5] END C=0.1, coef0=0, gamma=1, kernel=rbf;, score=0.705 total time=   0.0s
[CV 3/5] END C=0.1, coef0=0, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 4/5] END C=0.1, coef0=0, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 5/5] END C=0.1, coef0=0, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 1/5] END C=0.1, coef0=0, gamma=1, kernel=sigmoid;, score=0.733 total time=   0.0s
[CV 2/5] END C=0.1, coef0=0, gamma=1, kernel=sigmoid;, score=0.705 total time=   0.0s
[CV 3/5] END C=0.1, coef0=0, gamma=1, kernel=sigmoid;, score=0.682 total time=   0.0s
[CV 4/5] END C=0.1, coef0=0, gamma=1, kernel=sigmoid;, score=0.727 total time=   0.0s
[CV 5/5] END C=0.1, coef0=0, gamma=1, kernel=sigmoid;, score=0.773 total time=   0.0s
[CV 1/5] END C=0.1, coef0=0, gamma=1, kernel=poly;, score=0.667 total time=   0.0s
[CV 2/5] END 

[CV 1/5] END C=0.1, coef0=1, gamma=1, kernel=rbf;, score=0.711 total time=   0.0s
[CV 2/5] END C=0.1, coef0=1, gamma=1, kernel=rbf;, score=0.705 total time=   0.0s
[CV 3/5] END C=0.1, coef0=1, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 4/5] END C=0.1, coef0=1, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 5/5] END C=0.1, coef0=1, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 1/5] END C=0.1, coef0=1, gamma=1, kernel=sigmoid;, score=0.667 total time=   0.0s
[CV 2/5] END C=0.1, coef0=1, gamma=1, kernel=sigmoid;, score=0.727 total time=   0.0s
[CV 3/5] END C=0.1, coef0=1, gamma=1, kernel=sigmoid;, score=0.659 total time=   0.0s
[CV 4/5] END C=0.1, coef0=1, gamma=1, kernel=sigmoid;, score=0.705 total time=   0.0s
[CV 5/5] END C=0.1, coef0=1, gamma=1, kernel=sigmoid;, score=0.682 total time=   0.0s
[CV 1/5] END C=0.1, coef0=1, gamma=1, kernel=poly;, score=0.689 total time=   0.0s
[CV 2/5] END C=0.1, coef0=1, gamma=1, kernel=poly;, score=0.727 total time=  

[CV 4/5] END C=0.1, coef0=10, gamma=0.001, kernel=sigmoid;, score=0.727 total time=   0.0s
[CV 5/5] END C=0.1, coef0=10, gamma=0.001, kernel=sigmoid;, score=0.727 total time=   0.0s
[CV 1/5] END C=0.1, coef0=10, gamma=0.001, kernel=poly;, score=0.733 total time=   0.0s
[CV 2/5] END C=0.1, coef0=10, gamma=0.001, kernel=poly;, score=0.727 total time=   0.0s
[CV 3/5] END C=0.1, coef0=10, gamma=0.001, kernel=poly;, score=0.750 total time=   0.0s
[CV 4/5] END C=0.1, coef0=10, gamma=0.001, kernel=poly;, score=0.818 total time=   0.0s
[CV 5/5] END C=0.1, coef0=10, gamma=0.001, kernel=poly;, score=0.705 total time=   0.0s
[CV 1/5] END C=0.1, coef0=10, gamma=0.0001, kernel=rbf;, score=0.711 total time=   0.0s
[CV 2/5] END C=0.1, coef0=10, gamma=0.0001, kernel=rbf;, score=0.705 total time=   0.0s
[CV 3/5] END C=0.1, coef0=10, gamma=0.0001, kernel=rbf;, score=0.727 total time=   0.0s
[CV 4/5] END C=0.1, coef0=10, gamma=0.0001, kernel=rbf;, score=0.727 total time=   0.0s
[CV 5/5] END C=0.1, coef0=

[CV 3/5] END C=1, coef0=0.1, gamma=0.1, kernel=rbf;, score=0.773 total time=   0.0s
[CV 4/5] END C=1, coef0=0.1, gamma=0.1, kernel=rbf;, score=0.795 total time=   0.0s
[CV 5/5] END C=1, coef0=0.1, gamma=0.1, kernel=rbf;, score=0.818 total time=   0.0s
[CV 1/5] END C=1, coef0=0.1, gamma=0.1, kernel=sigmoid;, score=0.756 total time=   0.0s
[CV 2/5] END C=1, coef0=0.1, gamma=0.1, kernel=sigmoid;, score=0.659 total time=   0.0s
[CV 3/5] END C=1, coef0=0.1, gamma=0.1, kernel=sigmoid;, score=0.682 total time=   0.0s
[CV 4/5] END C=1, coef0=0.1, gamma=0.1, kernel=sigmoid;, score=0.659 total time=   0.0s
[CV 5/5] END C=1, coef0=0.1, gamma=0.1, kernel=sigmoid;, score=0.636 total time=   0.0s
[CV 1/5] END C=1, coef0=0.1, gamma=0.1, kernel=poly;, score=0.756 total time=   0.0s
[CV 2/5] END C=1, coef0=0.1, gamma=0.1, kernel=poly;, score=0.705 total time=   0.0s
[CV 3/5] END C=1, coef0=0.1, gamma=0.1, kernel=poly;, score=0.795 total time=   0.0s
[CV 4/5] END C=1, coef0=0.1, gamma=0.1, kernel=poly;,

[CV 2/5] END C=1, coef0=1, gamma=0.0001, kernel=poly;, score=0.705 total time=   0.0s
[CV 3/5] END C=1, coef0=1, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 4/5] END C=1, coef0=1, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 5/5] END C=1, coef0=1, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 1/5] END C=1, coef0=10, gamma=1, kernel=rbf;, score=0.733 total time=   0.0s
[CV 2/5] END C=1, coef0=10, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 3/5] END C=1, coef0=10, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 4/5] END C=1, coef0=10, gamma=1, kernel=rbf;, score=0.682 total time=   0.0s
[CV 5/5] END C=1, coef0=10, gamma=1, kernel=rbf;, score=0.750 total time=   0.0s
[CV 1/5] END C=1, coef0=10, gamma=1, kernel=sigmoid;, score=0.689 total time=   0.0s
[CV 2/5] END C=1, coef0=10, gamma=1, kernel=sigmoid;, score=0.705 total time=   0.0s
[CV 3/5] END C=1, coef0=10, gamma=1, kernel=sigmoid;, score=0.705 total time=   0

...

[CV 1/5] END C=100, coef0=0.1, gamma=0.0001, kernel=poly;, score=0.711 total time=   0.0s
[CV 2/5] END C=100, coef0=0.1, gamma=0.0001, kernel=poly;, score=0.705 total time=   0.0s
[CV 3/5] END C=100, coef0=0.1, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 4/5] END C=100, coef0=0.1, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 5/5] END C=100, coef0=0.1, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 1/5] END C=100, coef0=1, gamma=1, kernel=rbf;, score=0.733 total time=   0.0s
[CV 2/5] END C=100, coef0=1, gamma=1, kernel=rbf;, score=0.705 total time=   0.0s
[CV 3/5] END C=100, coef0=1, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 4/5] END C=100, coef0=1, gamma=1, kernel=rbf;, score=0.659 total time=   0.0s
[CV 5/5] END C=100, coef0=1, gamma=1, kernel=rbf;, score=0.750 total time=   0.0s
[CV 1/5] END C=100, coef0=1, gamma=1, kernel=sigmoid;, score=0.667 total time=   0.0s
[CV 2/5] END C=100, coef0=1, gamma=1, kernel=sigmoid;,

[CV 1/5] END C=100, coef0=10, gamma=0.1, kernel=poly;, score=0.667 total time=   0.0s
[CV 2/5] END C=100, coef0=10, gamma=0.1, kernel=poly;, score=0.659 total time=   0.1s
[CV 3/5] END C=100, coef0=10, gamma=0.1, kernel=poly;, score=0.705 total time=   0.2s
[CV 4/5] END C=100, coef0=10, gamma=0.1, kernel=poly;, score=0.568 total time=   0.1s
[CV 5/5] END C=100, coef0=10, gamma=0.1, kernel=poly;, score=0.727 total time=   0.2s
[CV 1/5] END C=100, coef0=10, gamma=0.01, kernel=rbf;, score=0.733 total time=   0.0s
[CV 2/5] END C=100, coef0=10, gamma=0.01, kernel=rbf;, score=0.750 total time=   0.0s
[CV 3/5] END C=100, coef0=10, gamma=0.01, kernel=rbf;, score=0.727 total time=   0.0s
[CV 4/5] END C=100, coef0=10, gamma=0.01, kernel=rbf;, score=0.750 total time=   0.0s
[CV 5/5] END C=100, coef0=10, gamma=0.01, kernel=rbf;, score=0.864 total time=   0.0s
[CV 1/5] END C=100, coef0=10, gamma=0.01, kernel=sigmoid;, score=0.711 total time=   0.0s
[CV 2/5] END C=100, coef0=10, gamma=0.01, kernel=s

[CV 3/5] END C=1000, coef0=0, gamma=0.0001, kernel=sigmoid;, score=0.682 total time=   0.0s
[CV 4/5] END C=1000, coef0=0, gamma=0.0001, kernel=sigmoid;, score=0.795 total time=   0.0s
[CV 5/5] END C=1000, coef0=0, gamma=0.0001, kernel=sigmoid;, score=0.705 total time=   0.0s
[CV 1/5] END C=1000, coef0=0, gamma=0.0001, kernel=poly;, score=0.711 total time=   0.0s
[CV 2/5] END C=1000, coef0=0, gamma=0.0001, kernel=poly;, score=0.705 total time=   0.0s
[CV 3/5] END C=1000, coef0=0, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 4/5] END C=1000, coef0=0, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 5/5] END C=1000, coef0=0, gamma=0.0001, kernel=poly;, score=0.727 total time=   0.0s
[CV 1/5] END C=1000, coef0=0.1, gamma=1, kernel=rbf;, score=0.733 total time=   0.0s
[CV 2/5] END C=1000, coef0=0.1, gamma=1, kernel=rbf;, score=0.705 total time=   0.0s
[CV 3/5] END C=1000, coef0=0.1, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 4/5] END C=1000, coe

[CV 1/5] END C=1000, coef0=1, gamma=0.01, kernel=sigmoid;, score=0.644 total time=   0.0s
[CV 2/5] END C=1000, coef0=1, gamma=0.01, kernel=sigmoid;, score=0.568 total time=   0.0s
[CV 3/5] END C=1000, coef0=1, gamma=0.01, kernel=sigmoid;, score=0.523 total time=   0.0s
[CV 4/5] END C=1000, coef0=1, gamma=0.01, kernel=sigmoid;, score=0.659 total time=   0.0s
[CV 5/5] END C=1000, coef0=1, gamma=0.01, kernel=sigmoid;, score=0.614 total time=   0.0s
[CV 1/5] END C=1000, coef0=1, gamma=0.01, kernel=poly;, score=0.711 total time=   0.0s
[CV 2/5] END C=1000, coef0=1, gamma=0.01, kernel=poly;, score=0.659 total time=   0.0s
[CV 3/5] END C=1000, coef0=1, gamma=0.01, kernel=poly;, score=0.705 total time=   0.0s
[CV 4/5] END C=1000, coef0=1, gamma=0.01, kernel=poly;, score=0.773 total time=   0.0s
[CV 5/5] END C=1000, coef0=1, gamma=0.01, kernel=poly;, score=0.864 total time=   0.0s
[CV 1/5] END C=1000, coef0=1, gamma=0.001, kernel=rbf;, score=0.778 total time=   0.0s
[CV 2/5] END C=1000, coef0=1

[CV 1/5] END C=1000, coef0=10, gamma=0.001, kernel=poly;, score=0.711 total time=   0.5s
[CV 2/5] END C=1000, coef0=10, gamma=0.001, kernel=poly;, score=0.727 total time=   0.0s
[CV 3/5] END C=1000, coef0=10, gamma=0.001, kernel=poly;, score=0.750 total time=   0.1s
[CV 4/5] END C=1000, coef0=10, gamma=0.001, kernel=poly;, score=0.750 total time=   0.1s
[CV 5/5] END C=1000, coef0=10, gamma=0.001, kernel=poly;, score=0.841 total time=   0.1s
[CV 1/5] END C=1000, coef0=10, gamma=0.0001, kernel=rbf;, score=0.756 total time=   0.0s
[CV 2/5] END C=1000, coef0=10, gamma=0.0001, kernel=rbf;, score=0.727 total time=   0.0s
[CV 3/5] END C=1000, coef0=10, gamma=0.0001, kernel=rbf;, score=0.682 total time=   0.0s
[CV 4/5] END C=1000, coef0=10, gamma=0.0001, kernel=rbf;, score=0.795 total time=   0.0s
[CV 5/5] END C=1000, coef0=10, gamma=0.0001, kernel=rbf;, score=0.705 total time=   0.0s
[CV 1/5] END C=1000, coef0=10, gamma=0.0001, kernel=sigmoid;, score=0.711 total time=   0.0s
[CV 2/5] END C=10

The SVM classification algorithm got results approximately the same as the work of kNN classifier on the given breast cancer dataset. The optimal hyperparameters found by GridSearchCV were C=1, gamma=0.01, and kernel='rbf'.  
What worked: 
- Preprocessing the data by dropping missing values and converting the class label to binary helped improve the model's performance. 
- GridSearchCV was able to find the optimal hyperparameters that improved the model's performance. 
What didn't work: 
- The dataset is relatively small, which could limit the model's ability to generalize to new data. 
- The SVM algorithm is sensitive to the choice of kernel function, and the 'rbf' kernel may not be the best choice for this dataset. 
What was expected: 
- It was expected that the SVM algorithm would be able to achieve high accuracy, recall, and precision on the breast cancer dataset, as SVMs are known to perform well on binary classification tasks. 
What was the result: 
- The SVM algorithm achieved an accuracy of 0.732, a recall of 0.263, and a precision of 0.833 on the breast cancer dataset. 
Conclusions: 
- The SVM algorithm is a powerful tool for binary classification tasks, and can achieve high performance on relatively small datasets, but in our case the performance isn't as high as we want.
- The choice of kernel function can have a significant impact on the model's performance, and it may be worth exploring other kernel functions for this dataset. 
Thoughts on how to improve the model for more accurate metrics: 
- Collecting more data could help improve the model's ability to generalize to new data. 
- Trying different kernel functions and hyperparameters could help improve the model's performance. 

In [156]:
# Define parameter grid for KernelDensity
param_grid = {'h': [0.1, 1, 10, 100, 1000]}

# Perform GridSearchCV to find optimal hyperparameters
best_acc = 0
best_h = None
for h in param_grid['h']:
    # Fit Parzen window classifier to training data
    pwc = ParzenWindowClassifier(h=h)
    pwc.fit(X_train, y_train)
    # Predict on test set
    y_pred = pwc.predict(X_test)
    # Calculate accuracy, recall, and precision
    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    pre = precision_score(y_test, y_pred)
    # Update best hyperparameters if accuracy improves
    if acc > best_acc:
        best_acc = acc
        best_h = h

# Print best hyperparameters
print("Best bandwidth:", best_h)

# Fit Parzen window classifier to training data using best hyperparameters
pwc = ParzenWindowClassifier(h=best_h)
pwc.fit(X_train, y_train)

# Predict on test set using best hyperparameters
y_pred = pwc.predict(X_test)

# Calculate accuracy, recall, and precision
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)

# Print metrics
print("Accuracy:", acc)
print("Recall:", rec)
print("Precision:", pre)

Best bandwidth: 10
Accuracy: 0.6785714285714286
Recall: 0.3157894736842105
Precision: 0.5454545454545454


The Parzen window classification algorithm was able to achieve relatively low accuracy, recall, and precision on the given breast cancer dataset. The optimal bandwidth found by GridSearchCV was 10.  
 
The Parzen window algorithm is a non-parametric classification algorithm that works by estimating the probability density function (PDF) of the training data for each class, and then using these PDFs to classify new data points.

To classify a new data point, the Parzen window algorithm first estimates the PDF of each class using a kernel function and a window size (also known as the bandwidth). The kernel function determines the shape of the PDF estimate, and the window size determines the smoothness of the estimate. The PDF estimate for each class is then evaluated at the location of the new data point, and the class with the highest PDF value is assigned to the new data point.

The Parzen window algorithm is best used when the underlying distribution of the data is not well understood, and when there are no strong assumptions about the shape of the decision boundary. It is also useful when the dataset is small, as it does not require a large number of parameters to be estimated. However, it can be computationally expensive, especially when the dimensionality of the data is high, as the number of PDF evaluations can grow rapidly with the number of dimensions.
 
Conclusions: 
- The choice of kernel function can have a significant impact on the model's performance, and it may be worth exploring other kernel functions for this dataset. 
- The Parzen window algorithm may not be the best choice for this dataset, as other classification algorithms like SVM and logistic regression were able to achieve higher performance. 
 
Thoughts on how to improve the model for more accurate metrics: 
- Collecting more data could help improve the model's ability to generalize to new data. 
- Trying different kernel functions and bandwidths could help improve the model's performance. 

Evaluate the Perceptron classifier. To find the optimal hyperparameters for the Perceptron classifier on the breast cancer dataset, we can perform a grid search over the hyperparameter space and evaluate the model using accuracy, recall, and precision metrics.

In [162]:
# Define the hyperparameter space
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_iterations': [100, 1000, 10000],
    'l1_penalty': [0, 0.1, 1],
    'l2_penalty': [0, 0.01, 0.1],
}

best_acc = 0

for lr in param_grid['learning_rate']:
    for n_iter in param_grid['n_iterations']:
        for l1 in param_grid['l1_penalty']:
            for l2 in param_grid['l2_penalty']:
                # Fit Perceptron classifier to training data
                perc = Perceptron(learning_rate=lr, n_iterations=n_iter, l1_penalty=l1, l2_penalty=l2)
                perc.fit(X_train, y_train)
                # Predict on test set
                y_pred = perc.predict(X_test)
                # Calculate accuracy, recall, and precision
                acc = accuracy_score(y_test, y_pred)
                rec = recall_score(y_test, y_pred)
                pre = precision_score(y_test, y_pred)
                # Update best hyperparameters if accuracy improves
                if acc > best_acc:
                    best_acc = acc
                    best_lr = lr
                    best_n_iter = n_iter
                    best_l1 = l1
                    best_l2 = l2

# Print best hyperparameters
print(f"Accuracy: {best_acc}, learning_rate: {best_lr}, n_iterations: {best_n_iter}, l1_penalty: {best_l1}, l2_penalty: {best_l2}")

# Fit Perceptron classifier to training data using best hyperparameters
perc = Perceptron(learning_rate=best_lr, n_iterations=best_n_iter, l1_penalty=best_l1, l2_penalty=best_l2)
perc.fit(X_train, y_train)

# Predict on test set using best hyperparameters
y_pred = perc.predict(X_test)

# Calculate accuracy, recall, and precision
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)

# Print metrics
print("Accuracy:", acc)
print("Recall:", rec)
print("Precision:", pre)

Accuracy: 0.3392857142857143, learning_rate: 0.01, n_iterations: 100, l1_penalty: 0, l2_penalty: 0
Accuracy: 0.3392857142857143
Recall: 1.0
Precision: 0.3392857142857143


Perceptron classification results are the worst among all 4 methods. The maximum accuracy value is only 0.339.
Essentially - a perceptron is a linear classifier that separates objects on a hyperplane and apply a nonlinear transformation at the end. Perceptron is best used for linearly separable datasets.

So, the reason of such low performance is small non-linearly separable dataset.

Some disadvantages of Perceptron method:
- Only works on linearly separable datasets.
- Can get stuck in a local minimum if the data is not linearly separable.
- Can be sensitive to the choice of learning rate.
To improve the work of the perceptron method, we need to try the following techniques:
- Use more advanced optimization algorithms, such as stochastic gradient descent or Adam, to update the weights and bias.
- Use different activation functions, such as the sigmoid or ReLU, to introduce non-linearity into the model.
- Extend the dataset.
- Use more advanced models, such as multi-layer neural networks, that can learn more complex decision boundaries.

Task 1 conslusion:

Best performances of classifiers:
- kNN: accuracy: 0.75, recall: 0.263, precision: 1;
- SVM: accuracy: 0.73, recall: 0.263, precision: 0.83;
- Parzen window: accuracy: 0.679, recall: 0.32, precision: 0.55;
- Perceptron: accuracy: 0.34, recall: 1.0, precision: 0.34.

In general, the choice of classification method depends on the characteristics of the dataset and the specific problem at hand. If the dataset is linearly separable, the perceptron method may be a good choice due to its simplicity and efficiency. If the dataset has non-linear decision boundaries, kNN, SVM, or Parzen window method may be better suited. kNN and Parzen window method are non-parametric and can handle non-linear decision boundaries, but can be computationally expensive during testing phase. SVM is also non-linear and robust to overfitting, but can be computationally expensive during training phase and sensitive to the choice of hyperparameters.

**Task 2 (40 points)**. 

Build machine learning models for the regression problem: namely, linear regression, the Nadarai-Watson method, and the Perceptron. 

Again, there are no restrictions on the dataset, you can take, for example: 

Servo Data Set : https://archive.ics.uci.edu/ml/datasets/Servo

Forest Fires Data Set : https://archive.ics.uci.edu/ml/datasets/Forest+Fires 

As for the requirements to the design and construction of the model and results, everything is the same as in the first task. 

We take Wine Quality (https://archive.ics.uci.edu/dataset/186/wine+quality) as a dataset. Two datasets are included, related to red and white vinho verde wine samples, from the north of Portugal. We will use the dataset with white wine (winequality-white).

In [269]:
path = "winequality-white.csv"
df = pd.read_csv(path, sep=';')

In [270]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Do little preprocessing.

In [280]:
# Create the target variable
y = df['quality']

# Create the feature matrix
X = df.drop(['quality'], axis=1)

# Normalize the data
X = (X - X.mean()) / X.std()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

Now, let's choose metrics for evaluating regression models on this dataset.

1. Mean Squared Error (MSE): It is a good metric for evaluating the overall performance of a model, and it is sensitive to large errors or outliers. 
2. Root Mean Squared Error (RMSE): This metric is the square root of the MSE and gives an idea of how far the predictions are from the true values in the same units as the dependent variable. It is a good metric for evaluating the overall performance of a model and is also sensitive to large errors or outliers. 
3. R-squared (R2): This metric measures how well the model fits the data and ranges from 0 to 1, with higher values indicating a better fit. It is a commonly used metric for regression tasks and can be used to compare models. R-squared is a good metric for evaluating the overall performance of a model, and it can also provide insight into the relationship between the independent and dependent variables.

In [272]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

Implement a multiple linear regression without using a third-party libraries. 
The main idea is:
1. The  fit  method initializes the weights to zero and runs a loop to update them using gradient descent. 
2. The  predict  method uses the learned weights to make predictions for new data. 

In [283]:
class MultipleLinearRegression:
    def __init__(self, learning_rate=0.01, iterations=1000, regularization=None, lambda_val=0.01):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.regularization = regularization
        self.lambda_val = lambda_val
    def fit(self, X, y):
        m, n = X.shape
        self.theta = np.zeros(n)
        self.cost_history = []
        for i in range(self.iterations):
            y_pred = X.dot(self.theta).ravel()
            error = np.subtract(y_pred, y)
            cost = np.sum(error ** 2) / (2 * m)
            if self.regularization == 'l1':
                regularization_term = self.lambda_val * np.sum(np.abs(self.theta))
            elif self.regularization == 'l2':
                regularization_term = self.lambda_val * np.sum(self.theta ** 2)
            else:
                regularization_term = 0
            cost += regularization_term
            self.cost_history.append(cost)
            gradient = X.T.dot(error) / m
            if self.regularization == 'l1':
                gradient += self.lambda_val * np.sign(self.theta)
            elif self.regularization == 'l2':
                gradient += self.lambda_val * self.theta
            self.theta -= self.learning_rate * gradient
    def predict(self, X):
        return X.dot(self.theta)

The hyperparameters are: 
 
-  learning_rate : The step size taken during gradient descent. 
-  iterations : The number of iterations to run gradient descent. 
-  regularization : The type of regularization to apply ( None ,  'l1' , or  'l2' ). 
-  lambda_val : The regularization parameter. 
 
These hyperparameters can be adjusted to improve the performance of the model on a given dataset.

Find an optimal hyper-parameters of this model using grid search over a range of parameters.

In [284]:
# Define the hyperparameter grid to search over
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'iterations': [500, 1000, 2000],
    'regularization': [None, 'l1', 'l2'],
    'lambda_val': [0.01, 0.1, 1.0]
}

In [285]:
best_mse = None

for lr in param_grid['learning_rate']:
    for iters in param_grid['iterations']:
        for reg in param_grid['regularization']:
            for lam in param_grid['lambda_val']:
                # Fit MultipleLinearRegression regressor to training data
                mlr = MultipleLinearRegression(learning_rate=lr, iterations=iters, regularization=reg, lambda_val=lam)
                mlr.fit(X_train, y_train)
                # Predict on test set
                y_pred = mlr.predict(X_test)
                # Calculate mse, rmse and r2
                mse = mean_squared_error(y_test, y_pred)
                rmse = np.sqrt(mse)
                r2 = r2_score(y_test, y_pred)
                # Update best hyperparameters if mse improves
                if (best_mse is None) or (mse < best_mse):
                    best_mse = mse
                    best_lr = lr
                    best_iters = iters
                    best_reg = reg
                    best_lam = lam
                    
# Print best hyperparameters
print(f"MSE: {best_mse}, learning_rate: {best_lr}, iterations: {best_iters}, regularization: {best_reg}, lambda_val: {best_lam}")

# Fit MultipleLinearRegression regressor to training data using best hyperparameters
mlr = MultipleLinearRegression(learning_rate=best_lr, iterations=best_iters, regularization=best_reg, lambda_val=best_lam)
mlr.fit(X_train, y_train)

# Predict on test set using best hyperparameters
y_pred = mlr.predict(X_test)

# Calculate accuracy, recall, and precision
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print metrics
print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)

MSE: 35.412863275056424, learning_rate: 0.01, iterations: 2000, regularization: l1, lambda_val: 0.1
MSE: 35.412863275056424
RMSE: 5.950870799728089
R2: -44.72510216315614


As we can see from the metrics, the result of the algorithm is extremely bad (R2 << 0). The R-squared (R2) score can be negative if the model performs worse than a model that always predicts the mean of the target variable. Perhaps our gradient is stuck at a local minimum. Let's try to change the gradient descent algorithm by adding momentum.

In [286]:
class MultipleLinearRegressionMomentum:
    def __init__(self, learning_rate=0.01, iterations=1000, regularization=None, lambda_val=0.01, momentum=0.9):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.regularization = regularization
        self.lambda_val = lambda_val
        self.momentum = momentum
    def fit(self, X, y):
        self.X = np.insert(X, 0, 1, axis=1)
        self.y = y.reshape(-1, 1)
        self.theta = np.zeros((self.X.shape[1], 1))
        self.v = np.zeros((self.X.shape[1], 1))
        for i in range(self.iterations):
            y_pred = self.X.dot(self.theta)
            error = y_pred - self.y
            if self.regularization == 'l1':
                regularization_term = self.lambda_val * np.sign(self.theta)
            elif self.regularization == 'l2':
                regularization_term = self.lambda_val * self.theta
            else:
                regularization_term = 0
            grad = (1 / len(self.X)) * self.X.T.dot(error) + regularization_term
            self.v = self.momentum * self.v + self.learning_rate * grad
            self.theta -= self.v
    def predict(self, X):
        X = np.insert(X, 0, 1, axis=1)
        return X.dot(self.theta).flatten()

In [290]:
# Define the hyperparameter grid to search over
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'iterations': [500, 1000, 2000],
    'regularization': [None, 'l1', 'l2'],
    'lambda_val': [0.01, 0.1, 1.0],
    'momentum' : [0.5, 0.9]
}

In [292]:
best_mse = None

for lr in param_grid['learning_rate']:
    for iters in param_grid['iterations']:
        for reg in param_grid['regularization']:
            for lam in param_grid['lambda_val']:
                for mom in param_grid['momentum']:
                    # Fit MultipleLinearRegressionMomentum regressor to training data
                    mlr = MultipleLinearRegressionMomentum(learning_rate=lr, iterations=iters, regularization=reg, lambda_val=lam, momentum=mom)
                    mlr.fit(X_train, y_train)
                    # Predict on test set
                    y_pred = mlr.predict(X_test)
                    # Calculate mse, rmse and r2
                    mse = mean_squared_error(y_test, y_pred)
                    rmse = np.sqrt(mse)
                    r2 = r2_score(y_test, y_pred)
                    # Update best hyperparameters if mse improves
                    if (best_mse is None) or (mse < best_mse):
                        best_mse = mse
                        best_lr = lr
                        best_iters = iters
                        best_reg = reg
                        best_lam = lam
                        best_mom = mom
                    
# Print best hyperparameters
print(f"MSE: {best_mse}, learning_rate: {best_lr}, iterations: {best_iters}, regularization: {best_reg}, lambda_val: {best_lam}, momentum: {best_mom}")

# Fit MultipleLinearRegressionMomentum regressor to training data using best hyperparameters
mlr = MultipleLinearRegressionMomentum(learning_rate=best_lr, iterations=best_iters, regularization=best_reg, lambda_val=best_lam, momentum=best_mom)
mlr.fit(X_train, y_train)

# Predict on test set using best hyperparameters
y_pred = mlr.predict(X_test)

# Calculate accuracy, recall, and precision
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print metrics
print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)

MSE: 0.5690247717229254, learning_rate: 0.1, iterations: 1000, regularization: None, lambda_val: 0.01, momentum: 0.9
MSE: 0.5690247717229254
RMSE: 0.7543373063311435
R2: 0.2652750042179155


As we can see, the results of the regression problem are significantly better (the metrics show that the regressor adequately copes with the task).

Some words about metrics:
The MSE measures the average squared difference between the predicted and true values, RMSE is the square root of MSE, while the R2 score measures the proportion of variance in the target variable that is explained by the independent variables in the model. 
 
To analyze the performance of the model, we can compare the values of the evaluation metrics to the range of possible values. For example, the range of possible values for the MSE and RMSE is [0, infinity), while the range of possible values for the R2 score is [-infinity, 1]. A lower value of MSE and RMSE indicates better performance, while a higher value of R2 score indicates better performance. 
 
From the results, we can see that the model performs reasonably well, with an MSE of around 0.57, RMSE of around 0.75, and an R2 score of around 0.265. However, there is still room for improvement, as the R2 score is relatively low, indicating that the model does not explain a significant amount of the variance in the target variable. 

We successfully tuning the hyperparameters of the model using grid search to find the best combination of hyperparameters that gives the best performance on the data.

To improve the model's performance, we could try various techniques such as feature selection, synthesizing new data for better training of the regressor, or using a more powerful algorithm such as a neural network, polynomial regression, decision trees or ensemble method.

In conclusion, linear regression is a commonly used method in machine learning for predicting a continuous target variable based on one or more input features. It is best to use linear regression when there is a linear relationship between the input features and the target variable. In other words, if the relationship between the input features and the target variable can be approximated by a straight line, then linear regression is a good choice of method. 

Nadaraya-Watson regression method

The Nadaraya-Watson method is a non-parametric regression technique that is used to estimate the conditional expectation of a target variable given the values of one or more input features. The method involves fitting a kernel function to the data and using it to estimate the conditional expectation at each point in the input space. The estimated function is a weighted average of the target variable values, with the weights determined by the kernel function and the distance between the input point and the data points. 
The basic steps of the Nadarai-Watson method are as follows:
 1. Choose a kernel function, such as the Gaussian kernel or the Epanechnikov kernel. The kernel function determines the shape of the weighting function used to estimate the conditional expectation.
 2. For each point in the input space, calculate the distance between the point and each of the data points.
 3. Apply the kernel function to each distance value to obtain a set of weights.
 4. Calculate the weighted average of the target variable values, using the weights obtained in step 3.
 5. Repeat steps 2-4 for each point in the input space to obtain the estimated function.
 
The Nadaraya-Watson method is a non-parametric method because it does not make any assumptions about the functional form of the relationship between the input features and the target variable. Instead, it estimates the conditional expectation using the data itself, without assuming any specific functional form. This makes it a flexible and powerful method for regression problems where the relationship between the input features and the target variable is complex and not well understood.

Implementation of Nadaraya-Watson regressor:

In [317]:
import numpy as np
class NadarayaWatsonRegressor:
    def __init__(self, kernel='gaussian', bandwidth=1.0):
        self.kernel = kernel
        self.bandwidth = bandwidth
    def fit(self, X, y):
        self.X = X
        self.y = y
    def predict(self, X_test):
        y_pred = []
        for x in X_test:
            k = self.kernel_function(x, self.X, self.bandwidth)
            y_pred.append(np.sum(np.dot(self.y, k)) / np.sum(k))
        return np.array(y_pred)
    def kernel_function(self, x, X, bandwidth):
        if self.kernel == 'gaussian':
            return np.exp(-0.5 * ((X - x) / bandwidth)**2) / np.sqrt(2 * np.pi * bandwidth**2)
        elif self.kernel == 'epanechnikov':
            return 3/4 * (1 - ((X - x) / bandwidth)**2) * (np.abs(X - x) <= bandwidth)
        else:
            raise ValueError('Invalid kernel function')

The NadarayaWatsonRegressor class has two hyperparameters, kernel and bandwidth.
 
The `kernel_function` method is used to compute the weights for each training example based on its distance to the test example. The weights are used to compute the weighted average of the target variables for the training examples, which is the predicted value for the test example. The kernel function determines the shape of the weight function and how it decays with distance.

The kernel function takes two arguments: `x`, which is the test example, and `X`, which is the training data. The `bandwidth` parameter determines the width of the kernel function and controls the degree of smoothing. If the bandwidth is small, the weight function will be more peaked and will assign a higher weight to training examples that are closer to the test example. If the bandwidth is large, the weight function will be smoother and will assign a more equal weight to all training examples, regardless of their distance to the test example.

In the code, we implemented two kernel functions: the Gaussian kernel and the Epanechnikov kernel. The Gaussian kernel assigns a weight to each training example based on its distance to the test example, with closer examples receiving a higher weight. The Epanechnikov kernel assigns a weight based on a parabolic function that is centered at the test example and has a bandwidth that determines the width of the function.

Now let's find the optimal hyperparameters for this regression with a grid search. 

In [318]:
# Define the hyperparameter grid to search over
param_grid = {
    'kernel': ['gaussian', 'epanechnikov'],
    'bandwidth': np.arange(0.1, 10.1, 0.1)
}

In [319]:
best_mse = None

for ker in param_grid['kernel']:
    for bw in param_grid['bandwidth']:
        # Fit NadarayaWatsonRegressor regressor to training data
        nwr = NadarayaWatsonRegressor(kernel=ker, bandwidth=bw)
        nwr.fit(X_train, y_train)
        # Predict on test set
        y_pred = nwr.predict(X_test)
        # Calculate mse, rmse and r2
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        # Update best hyperparameters if mse improves
        if (best_mse is None) or (mse < best_mse):
            best_mse = mse
            best_ker = ker
            best_bw = bw
                    
# Print best hyperparameters
print(f"MSE: {best_mse}, kernel: {best_ker}, bandwidth: {best_bw}")

# Fit NadarayaWatsonRegressor regressor to training data
nwr = NadarayaWatsonRegressor(kernel=best_ker, bandwidth=best_bw)
nwr.fit(X_train, y_train)

# Predict on test set using best hyperparameters
y_pred = nwr.predict(X_test)

# Calculate accuracy, recall, and precision
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print metrics
print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)

MSE: 0.6993552983106954, kernel: epanechnikov, bandwidth: 0.1
MSE: 0.6993552983106954
RMSE: 0.8362746548297966
R2: 0.09699218006680277


From the results, we can see that the Nadaraya-Watson regressor with a Epanechnikov kernel and bandwidth of 0.1 achieved the best performance in terms of MSE and RMSE. However, the R2 score is quite low, indicating that the model is not capturing much of the variance in the target variable.  
 
To improve the model, we could try feature engineering, ensemble methods (combining multiple Nadaraya-Watson regressors with different hyperparameters or even different models), non-parametric methods(k-nearest neighbors or decision trees).

Perceptron regressor

Implement Perceptron regressor from scratch:

In [322]:
class PerceptronRegressor:
    def __init__(self, learning_rate=0.01, n_epochs=100):
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        for _ in range(self.n_epochs):
            for i in range(n_samples):
                y_pred = np.dot(X[i], self.weights) + self.bias
                error = y[i] - y_pred
                self.weights += self.learning_rate * error * X[i]
                self.bias += self.learning_rate * error
        return self
    def predict(self, X):
        y_pred = np.dot(X, self.weights) + self.bias
        return y_pred

In this Perceptron regressor, there are two hyperparameters:
1. Learning rate: This hyperparameter controls the step size of the weight updates during training. A larger learning rate can lead to faster convergence, but may also cause the weights to oscillate or diverge. A smaller learning rate can lead to slower convergence, but may also result in better generalization to new data.
2. Number of epochs: This hyperparameter controls the number of times the entire training dataset is passed through during training. A larger number of epochs can lead to better convergence and potentially better performance, but may also lead to overfitting if the model is too complex or the dataset is too small. A smaller number of epochs can lead to faster training, but may result in underfitting if the model is not complex enough to capture the underlying patterns in the data.

As usual, we choose the optimal hyperparameters based on the grid search.

In [323]:
# Create a parameter grid for the learning rate and number of epochs hyperparameters
param_grid = {'learning_rate': np.arange(0.001, 0.101, 0.01), 'n_epochs': range(10, 101, 10)}

In [324]:
best_mse = None

for lr in param_grid['learning_rate']:
    for ep in param_grid['n_epochs']:
        # Fit PerceptronRegressor regressor to training data
        perc = PerceptronRegressor(learning_rate=lr, n_epochs=ep)
        perc.fit(X_train, y_train)
        # Predict on test set
        y_pred = perc.predict(X_test)
        # Calculate mse, rmse and r2
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        # Update best hyperparameters if mse improves
        if (best_mse is None) or (mse < best_mse):
            best_mse = mse
            best_lr = lr
            best_ep = ep
                    
# Print best hyperparameters
print(f"MSE: {best_mse}, learning_rate: {best_lr}, n_epochs: {best_ep}")

# Fit PerceptronRegressor regressor to training data
perc = PerceptronRegressor(learning_rate=best_lr, n_epochs=best_ep)
perc.fit(X_train, y_train)

# Predict on test set using best hyperparameters
y_pred = perc.predict(X_test)

# Calculate accuracy, recall, and precision
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print metrics
print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)

MSE: 0.5722770021423369, learning_rate: 0.001, n_epochs: 100
MSE: 0.5722770021423369
RMSE: 0.7564899220362006
R2: 0.2610757230970788


The results of choosing the optimal hyperparameters are as follows: best learning_rate=0.001, best n_epochs=100. With these hyperparameters the model achieved MSE=0.5723, RMSE=0.756, R2=0.261. The results replicate the results of Multiple Linear Regression.

Task 2 conclusions:

Best performances of regressors:
- Multiple linear regression: MSE: 0.57, RMSE: 0.75, R2: 0.27;
- Nadaraya-Watson: MSE: 0.7, RMSE: 0.84, R2: 0.1;
- Perceptron: MSE: 0.57, RMSE: 0.76, R2: 0.26.
 
In summary, the multiple linear regressor is a simple and interpretable linear regression algorithm that can handle multiple input features, but may be sensitive to outliers and multicollinearity. The Nadaraya-Watson regressor is a non-parametric regression algorithm that can handle non-linear relationships between the input features and target variable, but may be sensitive to the choice of kernel function and bandwidth parameter. The Perceptron regressor is a linear regression algorithm that uses a single layer neural network (so that's the reason of similarity of results between MultipleLinearRegressor and PerceptronRegressor) and can handle multiple input features, but may be prone to overfitting and sensitive to the choice of hyperparameters. The choice of algorithm depends on the specific problem at hand and the characteristics of the dataset.

**Task 3 (20 points)**.

The questions that make up the task are very common for interview questions. More often than not, they begin to go deeper into more narrow questions related to deep learning, so it is important to be able to answer them correctly to go further. Here it is important for me to read your reflections specifically: please don't use the internet. You can refresh your knowledge with our lecture and seminar, you can refer to articles on the Internet, but the answer itself should come only from your head, after re-reading the material. So try to write what you think, it will be immediately obvious.

*Explain in your own words how you understand the structure of neural networks.* 

My understanding of the structure of neural networks is that they are a type of machine learning model that is inspired by the structure and function of the human brain. Neural networks consist of layers of interconnected nodes, also known as neurons, which process and transmit information through the network. The input layer receives the input data, and the output layer produces the output predictions. In between the input and output layers, there can be one or more hidden layers, which perform intermediate computations on the input data. Each neuron in the network receives inputs from the previous layer, applies a non-linear mathematical function to the inputs, and passes the output to the next layer. The weights and biases of the neurons are learned during the training process, which involves adjusting the weights and biases to minimize the difference between the predicted output and the actual output. The structure of the neural network, including the number of layers and the number of neurons in each layer, can be adjusted to improve the performance of the model on a specific task.

*How does backward error propagation work?*

It is a common algorithm used for training artificial neural networks. The algorithm works by first passing the input data forward through the neural network to produce an output prediction. The difference between the predicted output and the actual output is then calculated using a loss function. The goal of the algorithm is to minimize this loss function by adjusting the weights and biases of the neurons in the network.  
 
The backward error propagation algorithm works by propagating the error backwards through the network from the output layer to the input layer. The error is used to update the weights and biases of the neurons in each layer, starting from the output layer and moving backwards towards the input layer. The amount of error that each neuron contributes to the output is calculated using the chain rule of calculus. This involves calculating the partial derivative of the loss function with respect to the output of each neuron, and then using this to calculate the partial derivative of the loss function with respect to the weights and biases of each neuron. 
 
The updated weights and biases are then used to repeat the forward pass through the network, and the process is repeated until the loss function is minimized. The backward error propagation algorithm is an iterative process that requires many iterations to converge to a minimum. The learning rate, which controls the step size of the weight updates, is an important hyperparameter that can affect the speed and accuracy of the training process.

*What are the advantages and disadvantages of this approach (backpropagation)?*

Advantages:
1. Since the algorithm uses first-order methods, it is very computationally efficient and can be parallelized to speed up the training process.
2. Allows to train neural networks with a large number of layers (deep). This is very important, as we know it is the depth, not the width of the neural network that allows us to solve complex problems.
3. The algorithm is flexible and can be used with a variety of activation functions and network architectures, allowing for customization to the specific problem at hand.

Disadvantages:
1. The algorithm can get stuck in local minima, where the loss function is minimized but not optimal, and may require multiple restarts to find the global minimum.
2. The algorithm requires a large amount of labeled training data to learn the weights and biases of the network, which can be time-consuming and expensive to obtain.
3. Backpropagation can be sensitive to the choice of hyperparameters, such as the learning rate and the number of hidden layers, which can affect the performance of the model.

*What does initialization of weights in neural networks mean?*

It is the process of setting the initial values of the weights and biases of the neurons in the network before training begins. 

Initialization of weights is crucial for network performance, since it sets the initial state of the algorithm in the state space. Therefore, if initial weights are chosen at local minima, the performance of algorithms is significantly reduced, since the optimization method may not lead the weights of the algorithm to global minima of the loss function. Also the learning time of the algorithm can significantly decrease for the same reason.

There are several methods for initializing the weights in a neural network, such as random initialization, Xavier initialization, and He initialization:
- Random initialization involves setting the weights and biases to random values drawn from a uniform or normal distribution. 
- Xavier initialization and He initialization are more sophisticated methods that take into account the number of neurons in the previous layer and the activation function used in the current layer to set the initial values of the weights and biases. These methods aim to prevent the vanishing or exploding gradient problem, which can occur when the gradients become too small or too large during training and can slow down or prevent learning. 

*What is the activation function needed for and which activation function do you think is the most suitable for deep learning tasks and why?*

It is a component of artificial neural networks that adds non-linearity to the output of each neuron. The activation function takes the weighted sum of the inputs to a neuron and applies a non-linear function to produce the output of the neuron. The choice of activation function can affect the performance of the network, as different functions have different properties and can be more or less suitable for different tasks. 

There are several activation functions in deep learning, such as sigmoid, tanh, ReLU, and softmax. 

ReLU (Rectified Linear Unit) is the most suitable for deep learning tasks. This is because ReLU is a simple and computationally efficient function that is easy to optimize and avoids the vanishing gradient problem that can occur with other activation functions. ReLU sets the output of a neuron to zero if the input is negative, and to the input value if the input is positive, which allows for efficient learning of sparse representations. However, ReLU can suffer from the "dying ReLU" problem, where neurons can become permanently inactive if their input is always negative. To avoid this problem, we can use Leaky ReLU or ELU, which have a small non-zero output for negative inputs.